# Home Credit Default Risk
<a id="ID_top"></a>

Use nbviewer to view an interactive version of the code [here](https://nbviewer.jupyter.org/github/milesin3d/HomeCreditDefaultRisk/blob/master/HomeCreditDefaultRisk.ipynb 'Click Here').

# Outline
---
- [1) Introduction](#ID_1) <br>
    - [1.1) Problem Statement](#ID_1.1) <br>
    - [1.2) Import Libraries](#ID_1.2) <br>

- [2) Data](#ID_2) <br>
    - [2.1) Read Files](#ID_2.1) <br>
    - [2.2 Descriptions](#ID_2.2) <br>
    - [2.3 Feature Types](#ID_2.3) <br>
    - [2.4 Missing Values](#ID_2.4) <br>
    - [2.5 Outlier Treatment](#ID_2.5) <br>
    - [2.6) Feature Engineering](#ID_2.6) <br>
    - [2.7) Encoding](#ID_2.7) <br>
        - [2.7.1) Label Encoding](#ID_2.7.1) <br>
        - [2.7.2) One-Hot Encoding](#ID_2.7.2) <br>
    - [2.8) Combine Datasets](#ID_2.8) <br>
    - [2.9) Balanced Dataset](#ID_2.9) <br>
    - [2.10) Split DateBalanced Dataset](#ID_2.10) <br>

- [3) Exploratory Data Analysts (EDA)](#ID_3) <br>
    - [3.1) Correlation Matrix](#ID_3.1) <br>

- [4) Variable Selection](#ID_4) <br>
    - [4.1) Information Value Analysis (IV)](#ID_4.1) <br>
    - [4.2) Variance Inflation Factor (VIF)](#ID_4.2) <br>
    - [4.3) Final Training and Test Sets](#ID_4.3) <br>

In [97]:
# how to create a internal link
# [title](#ID_) <br> # outline template
# <a id="ID_"></a> # title template
# [Back to Top](#ID_top) <br>

# 1) Introduction ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
<a id="ID_1"></a>
[Back to Top](#ID_top) <br>

## 1.1) Problem Statement
---
<a id="ID_1.1"></a>

Use alternative data to predict the repayment ability of people with insufficient or non-existent credit histories.

## 1.2) Import Libraries
---
<a id="ID_1.2"></a>

In [ ]:
# python packages used throughout the project

# importing and reading files
import chardet # check encoding for csv file
import os # for reading csv files
import glob # for reading csv files

# tabular data handling
import pandas as pd
import numpy as np

# plot data
import matplotlib.pyplot as plt
import seaborn as sns

import math

# warning package
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # suppress the warning globally

# 2) Data ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___ ___
<a id="ID_2"></a>
[Back to Top](#ID_top) <br>

## 2.1) Read Files
---
<a id="ID_2.1"></a>

In [4]:
# create a dictionary with file names and their abbreviations

# abbreviations for each data file
data_abb_names = ['a_test', 
                  'a_train', 
                  'b', 
                  'bb', 
                  'ccb', 
                  'hccd', 
                  'ip', 
                  'pcb', 
                  'pa', 
                  'ss']

# file path for csv files
path = r'data/raw'

# list of file names to be read
data_file_names = glob.glob(os.path.join(path, "*.csv")) 

# dictionary of file name and abbreviation
data_file_abb = dict(zip(data_file_names, data_abb_names))

# don't need to import description file
data_file_abb.pop('data/raw\\HomeCredit_columns_description.csv', None)

data_file_abb

{'data/raw\\application_test.csv': 'a_test',
 'data/raw\\application_train.csv': 'a_train',
 'data/raw\\bureau.csv': 'b',
 'data/raw\\bureau_balance.csv': 'bb',
 'data/raw\\credit_card_balance.csv': 'ccb',
 'data/raw\\installments_payments.csv': 'ip',
 'data/raw\\POS_CASH_balance.csv': 'pcb',
 'data/raw\\previous_application.csv': 'pa',
 'data/raw\\sample_submission.csv': 'ss'}

In [5]:
# use a for loop to import data from csv files

for k, v in data_file_abb.items():
    print(k) # print file name and location
    
    df_name = data_file_abb[k] # use the abbreviation to name the df
    
    vars()[df_name] = pd.read_csv(k, # file name
                                  header=0, # first row has the column headers
                                  index_col=0, # first column has the index values
                                  nrows=10000) # read only n rows


data/raw\application_test.csv
data/raw\application_train.csv
data/raw\bureau.csv
data/raw\bureau_balance.csv
data/raw\credit_card_balance.csv
data/raw\installments_payments.csv
data/raw\POS_CASH_balance.csv
data/raw\previous_application.csv
data/raw\sample_submission.csv


## 2.2) Descriptions
---
<a id="ID_2.2"></a>

- column descriptions
- preview 5 rows
- info
- describe

In [6]:
# read decription csv
# this file includes all the column headers for each of the files and a description for each

# check encoding for csv file
with open('data/raw\\HomeCredit_columns_description.csv', 'rb') as f:
    result = chardet.detect(f.read())
    
# read file
descrip = pd.read_csv('data/raw\\HomeCredit_columns_description.csv', 
                 index_col=0, 
                 encoding=result['encoding'])
descrip = descrip.replace(np.nan, '', regex=True)
descrip.head()

,Table,Row,Description,Special
1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,
2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,
5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,
6,application_{train|test}.csv,CODE_GENDER,Gender of the client,
7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,


### 2.2.1) Application_test|train

In [7]:
# column descriptions
descrip[descrip['Table']=='application_{train|test}.csv'][['Row','Description','Special']]

,Row,Description,Special
1,SK_ID_CURR,ID of loan in our sample,
2,TARGET,Target variable (1 - client with payment diffi...,
5,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,
6,CODE_GENDER,Gender of the client,
7,FLAG_OWN_CAR,Flag if the client owns a car,
8,FLAG_OWN_REALTY,Flag if client owns a house or flat,
9,CNT_CHILDREN,Number of children the client has,
10,AMT_INCOME_TOTAL,Income of the client,
11,AMT_CREDIT,Credit amount of the loan,
12,AMT_ANNUITY,Loan annuity,


In [8]:
# application_train.csv
print("The training data has %s loan applications, with %s features, plus the target variable." \
      % (a_train.shape[0],a_train.shape[1]-1))
a_train.head()

The training data has 10000 loan applications, with 120 features, plus the target variable.


,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
a_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 100002 to 111633
Columns: 121 entries, TARGET to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(40), object(16)
memory usage: 9.3+ MB


In [10]:
a_train.describe()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,10000.000000,10000.000000,1.000000e+04,1.000000e+04,10000.000000,9.993000e+03,10000.000000,10000.000000,10000.00000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,8634.000000,8634.000000,8634.000000,8634.000000,8634.000000,8634.000000
mean,0.077500,0.416300,1.674489e+05,6.007531e+05,27088.010550,5.403851e+05,0.020898,-16025.273100,62821.78480,-4986.483900,...,0.006900,0.000400,0.000500,0.000300,0.007297,0.008107,0.032893,0.274612,0.269748,1.889738
std,0.267397,0.722251,9.422260e+04,4.032643e+05,14384.946154,3.708941e+05,0.013909,4340.528828,140464.31594,3540.591931,...,0.082783,0.019997,0.022356,0.017319,0.087793,0.117620,0.193326,0.926332,0.614957,1.857018
min,0.000000,0.000000,2.565000e+04,4.500000e+04,2596.500000,4.500000e+04,0.000938,-25160.000000,-15632.00000,-20981.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.125000e+05,2.700000e+05,16434.000000,2.385000e+05,0.010006,-19595.250000,-2818.25000,-7491.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.440000e+05,5.130000e+05,24970.500000,4.500000e+05,0.018850,-15783.500000,-1224.00000,-4478.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,1.000000,2.025000e+05,8.100000e+05,34636.500000,6.840000e+05,0.028663,-12388.750000,-286.75000,-2004.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,1.000000,7.000000,1.935000e+06,2.961000e+06,135936.000000,2.961000e+06,0.072508,-7705.000000,365243.00000,0.000000,...,1.000000,1.000000,1.000000,1.000000,2.000000,4.000000,4.000000,16.000000,8.000000,13.000000


In [11]:
# application_test.csv
print("The testing data has %s loan applications, with %s features, with no target variable." % (a_test.shape[0],a_test.shape[1]))
a_test.head()

The testing data has 10000 loan applications, with 120 features, with no target variable.


,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
a_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 100001 to 172574
Columns: 120 entries, NAME_CONTRACT_TYPE to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(39), object(16)
memory usage: 9.2+ MB


In [13]:
a_test.describe()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,10000.000000,1.000000e+04,1.000000e+04,9990.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.0,10000.0,10000.0,8783.000000,8783.000000,8783.000000,8783.000000,8783.000000,8783.000000
mean,0.393700,1.782860e+05,5.134814e+05,29229.285586,4.593674e+05,0.021264,-16064.491800,66760.685300,-4975.574600,-3043.217700,...,0.001700,0.0,0.0,0.0,0.002846,0.002049,0.003074,0.008653,0.547535,1.953888
std,0.695808,1.119607e+05,3.605264e+05,15801.337507,3.324379e+05,0.014263,4337.538665,143785.156987,3524.712451,1574.693963,...,0.041198,0.0,0.0,0.0,0.053279,0.050009,0.057383,0.098579,0.704437,1.821339
min,0.000000,2.700000e+04,4.500000e+04,2295.000000,4.500000e+04,0.000533,-25154.000000,-17463.000000,-22842.000000,-6315.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.125000e+05,2.606400e+05,17812.125000,2.250000e+05,0.010006,-19651.250000,-2887.000000,-7466.750000,-4447.250000,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.575000e+05,4.500000e+05,25971.750000,3.825000e+05,0.018850,-15799.000000,-1287.500000,-4528.500000,-3236.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,1.000000,2.250000e+05,6.750000e+05,37375.875000,6.165000e+05,0.028663,-12467.750000,-309.000000,-1930.000000,-1699.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000
max,8.000000,4.410000e+06,2.160000e+06,180576.000000,2.160000e+06,0.072508,-7725.000000,365243.000000,0.000000,0.000000,...,1.000000,0.0,0.0,0.0,1.000000,2.000000,2.000000,3.000000,5.000000,13.000000


### <font color='black'>2.2.2) Bureau</font>

In [14]:
descrip[descrip['Table']=='bureau.csv'][['Row','Description','Special']]

,Row,Description,Special
125,SK_ID_CURR,ID of loan in our sample - one loan in our sam...,hashed
126,SK_BUREAU_ID,Recoded ID of previous Credit Bureau credit re...,hashed
127,CREDIT_ACTIVE,Status of the Credit Bureau (CB) reported credits,
128,CREDIT_CURRENCY,Recoded currency of the Credit Bureau credit,recoded
129,DAYS_CREDIT,How many days before current application did c...,time only relative to the application
130,CREDIT_DAY_OVERDUE,Number of days past due on CB credit at the ti...,
131,DAYS_CREDIT_ENDDATE,Remaining duration of CB credit (in days) at t...,time only relative to the application
132,DAYS_ENDDATE_FACT,Days since CB credit ended at the time of appl...,time only relative to the application
133,AMT_CREDIT_MAX_OVERDUE,Maximal amount overdue on the Credit Bureau cr...,
134,CNT_CREDIT_PROLONG,How many times was the Credit Bureau credit pr...,


In [15]:
# bureau.csv
b.head()

,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
SK_ID_CURR,,,,,,,,,,,,,,,,
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [16]:
b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 215354 to 215641
Data columns (total 16 columns):
SK_ID_BUREAU              10000 non-null int64
CREDIT_ACTIVE             10000 non-null object
CREDIT_CURRENCY           10000 non-null object
DAYS_CREDIT               10000 non-null int64
CREDIT_DAY_OVERDUE        10000 non-null int64
DAYS_CREDIT_ENDDATE       9369 non-null float64
DAYS_ENDDATE_FACT         6229 non-null float64
AMT_CREDIT_MAX_OVERDUE    3595 non-null float64
CNT_CREDIT_PROLONG        10000 non-null int64
AMT_CREDIT_SUM            10000 non-null float64
AMT_CREDIT_SUM_DEBT       8460 non-null float64
AMT_CREDIT_SUM_LIMIT      6550 non-null float64
AMT_CREDIT_SUM_OVERDUE    10000 non-null float64
CREDIT_TYPE               10000 non-null object
DAYS_CREDIT_UPDATE        10000 non-null int64
AMT_ANNUITY               1834 non-null float64
dtypes: float64(8), int64(5), object(3)
memory usage: 1.3+ MB


In [17]:
b.describe()

,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
count,1.000000e+04,10000.000000,10000.000000,9369.000000,6229.000000,3595.000000,10000.00000,1.000000e+04,8.460000e+03,6550.000000,10000.000000,10000.000000,1.834000e+03
mean,5.720532e+06,-1153.844600,2.352000,455.315188,-1029.951999,3605.774670,0.00720,3.630334e+05,1.355542e+05,6605.021906,90.453047,-595.973900,1.842148e+04
std,3.488276e+03,799.514416,64.941218,4707.763029,725.398701,23736.010432,0.10559,9.682721e+05,6.151356e+05,41559.211865,4333.423801,777.218523,1.237030e+05
min,5.714462e+06,-2922.000000,0.000000,-2807.000000,-2820.000000,0.000000,0.00000,0.000000e+00,-1.323248e+04,-110293.605000,0.000000,-41857.000000,0.000000e+00
25%,5.717553e+06,-1686.250000,0.000000,-1152.000000,-1520.000000,0.000000,0.00000,5.350050e+04,0.000000e+00,0.000000,0.000000,-900.000000,0.000000e+00
50%,5.720470e+06,-1005.000000,0.000000,-310.000000,-889.000000,0.000000,0.00000,1.350000e+05,0.000000e+00,0.000000,0.000000,-393.500000,2.745000e+03
75%,5.723514e+06,-477.000000,0.000000,512.000000,-425.000000,0.000000,0.00000,3.150000e+05,3.977550e+04,0.000000,0.000000,-35.000000,1.583438e+04
max,5.726631e+06,-2.000000,2625.000000,31196.000000,-3.000000,822038.895000,4.00000,4.500000e+07,1.738731e+07,765000.000000,349428.060000,0.000000,4.358508e+06


### <font color='green'>2.2.3) Bureau Balance</font>

In [18]:
descrip[descrip['Table']=='bureau_balance.csv'][['Row','Description','Special']]

,Row,Description,Special
142,SK_BUREAU_ID,Recoded ID of Credit Bureau credit (unique cod...,hashed
143,MONTHS_BALANCE,Month of balance relative to application date ...,time only relative to the application
144,STATUS,Status of Credit Bureau loan during the month ...,


In [19]:
# bureau_balance.csv
bb.head()

,MONTHS_BALANCE,STATUS
SK_ID_BUREAU,,
5715448,0,C
5715448,-1,C
5715448,-2,C
5715448,-3,C
5715448,-4,C


In [20]:
bb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 5715448 to 5720034
Data columns (total 2 columns):
MONTHS_BALANCE    10000 non-null int64
STATUS            10000 non-null object
dtypes: int64(1), object(1)
memory usage: 234.4+ KB


In [21]:
bb.describe()

,MONTHS_BALANCE
count,10000.000000
mean,-27.670700
std,22.429279
min,-96.000000
25%,-41.000000
50%,-22.000000
75%,-10.000000
max,0.000000


### 2.2.4) Credit Card Balance
## <font color='green'>1.0) Title</font>

In [22]:
descrip[descrip['Table']=='credit_card_balance.csv'][['Row','Description','Special']]

,Row,Description,Special
153,SK_ID_PREV,ID of previous credit in Home credit related t...,hashed
154,SK_ID_CURR,ID of loan in our sample,hashed
155,MONTHS_BALANCE,Month of balance relative to application date ...,time only relative to the application
156,AMT_BALANCE,Balance during the month of previous credit,
157,AMT_CREDIT_LIMIT_ACTUAL,Credit card limit during the month of the prev...,
158,AMT_DRAWINGS_ATM_CURRENT,Amount drawing at ATM during the month of the ...,
159,AMT_DRAWINGS_CURRENT,Amount drawing during the month of the previou...,
160,AMT_DRAWINGS_OTHER_CURRENT,Amount of other drawings during the month of t...,
161,AMT_DRAWINGS_POS_CURRENT,Amount drawing or buying goods during the mont...,
162,AMT_INST_MIN_REGULARITY,Minimal installment for this month of the prev...,


In [23]:
# credit_card_balance.csv
ccb.head()

,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
SK_ID_PREV,,,,,,,,,,,,,,,,,,,,,
2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [24]:
ccb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 2562384 to 1669462
Data columns (total 22 columns):
SK_ID_CURR                    10000 non-null int64
MONTHS_BALANCE                10000 non-null int64
AMT_BALANCE                   10000 non-null float64
AMT_CREDIT_LIMIT_ACTUAL       10000 non-null int64
AMT_DRAWINGS_ATM_CURRENT      7933 non-null float64
AMT_DRAWINGS_CURRENT          10000 non-null float64
AMT_DRAWINGS_OTHER_CURRENT    7933 non-null float64
AMT_DRAWINGS_POS_CURRENT      7933 non-null float64
AMT_INST_MIN_REGULARITY       9674 non-null float64
AMT_PAYMENT_CURRENT           7952 non-null float64
AMT_PAYMENT_TOTAL_CURRENT     10000 non-null float64
AMT_RECEIVABLE_PRINCIPAL      10000 non-null float64
AMT_RECIVABLE                 10000 non-null float64
AMT_TOTAL_RECEIVABLE          10000 non-null float64
CNT_DRAWINGS_ATM_CURRENT      7933 non-null float64
CNT_DRAWINGS_CURRENT          10000 non-null int64
CNT_DRAWINGS_OTHER_CURRENT    7933 non-null float

In [25]:
ccb.describe()

,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,...,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF
count,10000.000000,10000.000000,10000.000000,1.000000e+04,7.933000e+03,1.000000e+04,7933.00000,7933.000000,9674.000000,7952.000000,...,10000.000000,10000.000000,10000.000000,7933.000000,10000.000000,7933.000000,7933.000000,9674.000000,10000.000000,10000.000000
mean,277510.860700,-24.022600,82718.755632,1.851525e+05,6.503634e+03,8.753622e+03,333.67517,4180.618867,4549.747072,13954.050243,...,79558.344607,82324.257300,82327.619106,0.342367,0.976700,0.005042,0.883777,17.833575,5.384200,0.168400
std,103038.193913,30.008928,140678.377574,1.913274e+05,2.976681e+04,3.546678e+04,6846.00085,23681.169662,7115.984951,44031.671988,...,135829.180157,140066.132731,140067.046788,1.074397,3.800684,0.079235,4.009709,19.027901,80.732815,9.964626
min,100048.000000,-95.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,188619.750000,-52.000000,0.000000,4.500000e+04,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000,147.915000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000
50%,278904.500000,-7.000000,0.000000,1.350000e+05,0.000000e+00,0.000000e+00,0.00000,0.000000,0.000000,4500.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.000000,0.000000,0.000000
75%,365832.500000,-4.000000,126663.401250,2.250000e+05,0.000000e+00,0.000000e+00,0.00000,0.000000,6964.200000,11700.000000,...,121756.691250,126080.887500,126080.887500,0.000000,0.000000,0.000000,0.000000,25.000000,0.000000,0.000000
max,456247.000000,-1.000000,940575.195000,1.125000e+06,1.237500e+06,1.237500e+06,234000.00000,849442.320000,46728.540000,916812.000000,...,899968.590000,933656.625000,933656.625000,19.000000,101.000000,3.000000,101.000000,116.000000,2192.000000,701.000000


### 2.2.5) Installments Payments

In [26]:
descrip[descrip['Table']=='installments_payments.csv'][['Row','Description','Special']]

,Row,Description,Special
214,SK_ID_PREV,ID of previous credit in Home credit related t...,hashed
215,SK_ID_CURR,ID of loan in our sample,hashed
216,NUM_INSTALMENT_VERSION,Version of installment calendar (0 is for cred...,
217,NUM_INSTALMENT_NUMBER,On which installment we observe payment,
218,DAYS_INSTALMENT,When the installment of previous credit was su...,time only relative to the application
219,DAYS_ENTRY_PAYMENT,When was the installments of previous credit p...,time only relative to the application
220,AMT_INSTALMENT,What was the prescribed installment amount of ...,
221,AMT_PAYMENT,What the client actually paid on previous cred...,


In [27]:
# installments_payments.csv
ip.head()

,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
SK_ID_PREV,,,,,,,
1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [28]:
ip.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1054186 to 2305655
Data columns (total 7 columns):
SK_ID_CURR                10000 non-null int64
NUM_INSTALMENT_VERSION    10000 non-null float64
NUM_INSTALMENT_NUMBER     10000 non-null int64
DAYS_INSTALMENT           10000 non-null float64
DAYS_ENTRY_PAYMENT        10000 non-null float64
AMT_INSTALMENT            10000 non-null float64
AMT_PAYMENT               10000 non-null float64
dtypes: float64(5), int64(2)
memory usage: 625.0 KB


In [29]:
ip.describe()

,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
count,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,1.000000e+04,1.000000e+04
mean,150279.256800,0.86030,19.096900,-1044.528200,-1053.228500,1.792817e+04,1.820436e+04
std,29053.099056,1.09759,27.175047,802.562961,802.297663,6.217250e+04,6.690229e+04
min,100009.000000,0.00000,1.000000,-2920.000000,-2976.000000,4.500000e-02,0.000000e+00
25%,125158.000000,0.00000,4.000000,-1666.000000,-1674.000000,4.325648e+03,3.461332e+03
50%,150075.000000,1.00000,8.000000,-820.000000,-831.000000,8.992125e+03,8.144010e+03
75%,175930.500000,1.00000,19.000000,-357.750000,-366.000000,1.677538e+04,1.610090e+04
max,199986.000000,34.00000,210.000000,-2.000000,-2.000000,2.292737e+06,2.292737e+06


### 2.2.6) POS Cash Balance

In [30]:
descrip[descrip['Table']=='POS_CASH_balance.csv'][['Row','Description','Special']]

,Row,Description,Special
145,SK_ID_PREV,ID of previous credit in Home Credit related t...,
146,SK_ID_CURR,ID of loan in our sample,
147,MONTHS_BALANCE,Month of balance relative to application date ...,time only relative to the application
148,CNT_INSTALMENT,Term of previous credit (can change over time),
149,CNT_INSTALMENT_FUTURE,Installments left to pay on the previous credit,
150,NAME_CONTRACT_STATUS,Contract status during the month,
151,SK_DPD,DPD (days past due) during the month of previo...,
152,SK_DPD_DEF,DPD during the month with tolerance (debts wit...,


In [31]:
# POS_CASH_balance.csv
pcb.head()

,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
SK_ID_PREV,,,,,,,
1803195,182943,-31,48.0,45.0,Active,0,0
1715348,367990,-33,36.0,35.0,Active,0,0
1784872,397406,-32,12.0,9.0,Active,0,0
1903291,269225,-35,48.0,42.0,Active,0,0
2341044,334279,-35,36.0,35.0,Active,0,0


In [32]:
pcb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1803195 to 1050869
Data columns (total 7 columns):
SK_ID_CURR               10000 non-null int64
MONTHS_BALANCE           10000 non-null int64
CNT_INSTALMENT           9986 non-null float64
CNT_INSTALMENT_FUTURE    9986 non-null float64
NAME_CONTRACT_STATUS     10000 non-null object
SK_DPD                   10000 non-null int64
SK_DPD_DEF               10000 non-null int64
dtypes: float64(2), int64(4), object(1)
memory usage: 625.0+ KB


In [33]:
pcb.describe()

,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF
count,10000.00000,10000.000000,9986.000000,9986.000000,10000.000000,10000.000000
mean,279859.41680,-30.453300,19.155217,11.431104,0.687900,0.069600
std,102493.92398,16.820699,11.696245,11.677717,19.736901,1.203786
min,100028.00000,-96.000000,1.000000,0.000000,0.000000,0.000000
25%,192079.50000,-40.000000,12.000000,3.000000,0.000000,0.000000
50%,280158.50000,-34.000000,14.000000,8.000000,0.000000,0.000000
75%,368805.00000,-17.000000,24.000000,15.000000,0.000000,0.000000
max,456242.00000,-1.000000,60.000000,60.000000,1105.000000,95.000000


### 2.2.7) Previous Application

In [34]:
descrip[descrip['Table']=='previous_application.csv'][['Row','Description','Special']]

,Row,Description,Special
176,SK_ID_PREV,ID of previous credit in Home credit related t...,hashed
177,SK_ID_CURR,ID of loan in our sample,hashed
178,NAME_CONTRACT_TYPE,"Contract product type (Cash loan, consumer loa...",
179,AMT_ANNUITY,Annuity of previous application,
180,AMT_APPLICATION,For how much credit did client ask on the prev...,
181,AMT_CREDIT,Final credit amount on the previous applicatio...,
182,AMT_DOWN_PAYMENT,Down payment on the previous application,
183,AMT_GOODS_PRICE,Goods price of good that client asked for (if ...,
184,WEEKDAY_APPR_PROCESS_START,On which day of the week did the client apply ...,
185,HOUR_APPR_PROCESS_START,Approximately at what day hour did the client ...,rounded


In [35]:
# previous_application.csv
pa.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
SK_ID_PREV,,,,,,,,,,,,,,,,,,,,,
2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
pa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 2030495 to 2744934
Data columns (total 36 columns):
SK_ID_CURR                     10000 non-null int64
NAME_CONTRACT_TYPE             10000 non-null object
AMT_ANNUITY                    7915 non-null float64
AMT_APPLICATION                10000 non-null float64
AMT_CREDIT                     10000 non-null float64
AMT_DOWN_PAYMENT               4954 non-null float64
AMT_GOODS_PRICE                7878 non-null float64
WEEKDAY_APPR_PROCESS_START     10000 non-null object
HOUR_APPR_PROCESS_START        10000 non-null int64
FLAG_LAST_APPL_PER_CONTRACT    10000 non-null object
NFLAG_LAST_APPL_IN_DAY         10000 non-null int64
RATE_DOWN_PAYMENT              4954 non-null float64
RATE_INTEREST_PRIMARY          37 non-null float64
RATE_INTEREST_PRIVILEGED       37 non-null float64
NAME_CASH_LOAN_PURPOSE         10000 non-null object
NAME_CONTRACT_STATUS           10000 non-null object
DAYS_DECISION                  10000 non

In [37]:
pa.describe()

,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
count,10000.00000,7915.000000,1.000000e+04,1.000000e+04,4954.000000,7.878000e+03,10000.000000,10000.000000,4954.000000,37.000000,37.000000,10000.00000,1.000000e+04,7915.000000,6144.000000,6144.000000,6144.000000,6144.000000,6144.000000,6144.000000
mean,279198.86100,15340.900503,1.680960e+05,1.891482e+05,6374.445524,2.133911e+05,12.532700,0.997000,0.078541,0.209091,0.793997,-903.02830,7.471178e+02,15.606822,344026.308919,14753.019043,31949.402832,76787.306315,81746.756510,0.323730
std,102698.20804,14058.414356,2.772339e+05,3.047238e+05,16261.452968,2.964850e+05,3.306512,0.054693,0.107804,0.119227,0.092466,789.29142,4.002268e+04,13.969634,85556.607401,74569.436599,104310.869841,149811.921815,153165.387447,0.467937
min,100009.00000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,1.000000,0.000000,0.000000,0.142440,0.568710,-2920.00000,-1.000000e+00,0.000000,-2888.000000,-2889.000000,-2783.000000,-2829.000000,-2760.000000,0.000000
25%,191255.50000,6224.985000,2.245500e+04,2.669850e+04,0.000000,5.049338e+04,10.000000,1.000000,0.000000,0.160716,0.715645,-1348.25000,-1.000000e+00,6.000000,365243.000000,-1626.000000,-1277.250000,-1328.000000,-1289.000000,0.000000
50%,277700.00000,10842.885000,7.242525e+04,8.100000e+04,4.410000,1.027350e+05,12.000000,1.000000,0.000273,0.189136,0.835095,-600.00000,1.000000e+01,12.000000,365243.000000,-812.500000,-363.500000,-534.000000,-498.500000,0.000000
75%,368996.00000,19679.107500,1.800000e+05,1.985366e+05,7650.000000,2.250000e+05,15.000000,1.000000,0.108909,0.193330,0.852537,-294.00000,1.000000e+02,18.000000,365243.000000,-399.750000,120.000000,-65.000000,-38.000000,1.000000
max,456254.00000,204635.790000,2.925000e+06,3.020760e+06,489600.000000,2.925000e+06,22.000000,1.000000,0.818315,0.696163,0.867336,-2.00000,4.000000e+06,60.000000,365243.000000,365243.000000,365243.000000,365243.000000,365243.000000,1.000000


### 2.2.8) Sample Submission

Below is a sample submission for this project. The output includes the SK_ID_CURR and the associated TARGET output between 0 and 1.

In [38]:
# sample submission
ss.head()

,TARGET
SK_ID_CURR,
100001,0.5
100005,0.5
100013,0.5
100028,0.5
100038,0.5


In [39]:
ss.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 100001 to 172574
Data columns (total 1 columns):
TARGET    10000 non-null float64
dtypes: float64(1)
memory usage: 156.2 KB


In [40]:
ss.describe()

,TARGET
count,10000.0
mean,0.5
std,0.0
min,0.5
25%,0.5
50%,0.5
75%,0.5
max,0.5


## 2.3) Feature Types
<a id="ID_2.3"></a>

### 2.3.1) Application_test|train

- create an 'TRAIN_TEST' column in a_train and give the value 'train'
- create an 'TRAIN_TEST' column in a_train and give the value 'test'
- combine train and test datasets
- this way all the cleaning can be done at once, and then split at the end
- split data back into train/test df's using the 'TRAIN_TEST' column and remove the column

In [41]:
# combine a_train and a_test dfs
# clean data
# split back into a_train and a_test dfs

# create a 'TRAIN_TEST' column in a_train and give the value 'train'
a_train['TRAIN_TEST'] = 'train'

# create an 'TRAIN_TEST' column in a_train and give the value 'test'
a_test['TRAIN_TEST'] = 'test'

# stack train and test dfs
a = pd.concat([a_train, a_test])

print('a_train:', len(a_train))
print('a_test:',len(a_test))
print('a:',len(a))

a_train: 10000
a_test: 10000
a: 20000


In [42]:
a.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,TOTALAREA_MODE,TRAIN_TEST,WALLSMATERIAL_MODE,WEEKDAY_APPR_PROCESS_START,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_AVG,YEARS_BUILD_MEDI,YEARS_BUILD_MODE
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0149,train,"Stone, brick",WEDNESDAY,0.9722,0.9722,0.9722,0.6192,0.6243,0.6341
100003,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0714,train,Block,MONDAY,0.9851,0.9851,0.9851,0.7960,0.7987,0.8040
100004,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,train,NaN,MONDAY,NaN,NaN,NaN,NaN,NaN,NaN
100006,29686.5,312682.5,297000.0,135000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,train,NaN,WEDNESDAY,NaN,NaN,NaN,NaN,NaN,NaN
100007,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,train,NaN,THURSDAY,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
#list the columns
a_cols = list(a.columns)

# create a 'to-do' list of all the columns and remove them as they are reviewed
cols_to_clean = a_cols
len(cols_to_clean)

122

#### Boolean Columns (0,1)

In [44]:
# find all FLAG columns using their column headers
flag_cols = [x for x in a_cols if 'FLAG' in x]
flag_cols

['FLAG_CONT_MOBILE',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_17',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_20',
 'FLAG_DOCUMENT_21',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_EMAIL',
 'FLAG_EMP_PHONE',
 'FLAG_MOBIL',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'FLAG_PHONE',
 'FLAG_WORK_PHONE']

In [45]:
# use the descriptions to look for more FLAG columns

# return a list of the descriptions for each of the cols
flag_cols_2 = list(descrip[descrip['Table']=='application_{train|test}.csv'].Description)

# convert all strings in list to lowercase
flag_cols_2 = [x.lower() for x in flag_cols_2]

# if 'flag' is in the string, keep it
flag_cols_2 = [x for x in flag_cols_2 if 'flag' in x]

flag_cols_2

['flag if the client owns a car',
 'flag if client owns a house or flat',
 "flag if client's permanent address does not match contact address (1=different, 0=same, at region level)",
 "flag if client's permanent address does not match work address (1=different, 0=same, at region level)",
 "flag if client's contact address does not match work address (1=different, 0=same, at region level)",
 "flag if client's permanent address does not match contact address (1=different, 0=same, at city level)",
 "flag if client's permanent address does not match work address (1=different, 0=same, at city level)",
 "flag if client's contact address does not match work address (1=different, 0=same, at city level)"]

In [46]:
# these are the matching col headers from the cell before
flag_cols_2 = ['TARGET',
               'REG_REGION_NOT_LIVE_REGION',
               'REG_REGION_NOT_WORK_REGION',
               'LIVE_REGION_NOT_WORK_REGION',
               'REG_CITY_NOT_LIVE_CITY',
               'REG_CITY_NOT_WORK_CITY',
               'LIVE_CITY_NOT_WORK_CITY', 
               'EMERGENCYSTATE_MODE'
               ]

In [47]:
flag_cols += flag_cols_2
flag_cols

['FLAG_CONT_MOBILE',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_17',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_20',
 'FLAG_DOCUMENT_21',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_EMAIL',
 'FLAG_EMP_PHONE',
 'FLAG_MOBIL',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'FLAG_PHONE',
 'FLAG_WORK_PHONE',
 'TARGET',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'EMERGENCYSTATE_MODE']

In [48]:
# print columns and their unique values

for col in flag_cols:
    print(col, ':', a[col].unique())

FLAG_CONT_MOBILE : [1 0]
FLAG_DOCUMENT_10 : [0]
FLAG_DOCUMENT_11 : [0 1]
FLAG_DOCUMENT_12 : [0]
FLAG_DOCUMENT_13 : [0 1]
FLAG_DOCUMENT_14 : [0 1]
FLAG_DOCUMENT_15 : [0 1]
FLAG_DOCUMENT_16 : [0 1]
FLAG_DOCUMENT_17 : [0 1]
FLAG_DOCUMENT_18 : [0 1]
FLAG_DOCUMENT_19 : [0 1]
FLAG_DOCUMENT_2 : [0]
FLAG_DOCUMENT_20 : [0 1]
FLAG_DOCUMENT_21 : [0 1]
FLAG_DOCUMENT_3 : [1 0]
FLAG_DOCUMENT_4 : [0 1]
FLAG_DOCUMENT_5 : [0 1]
FLAG_DOCUMENT_6 : [0 1]
FLAG_DOCUMENT_7 : [0 1]
FLAG_DOCUMENT_8 : [0 1]
FLAG_DOCUMENT_9 : [0 1]
FLAG_EMAIL : [0 1]
FLAG_EMP_PHONE : [1 0]
FLAG_MOBIL : [1]
FLAG_OWN_CAR : ['N' 'Y']
FLAG_OWN_REALTY : ['Y' 'N']
FLAG_PHONE : [1 0]
FLAG_WORK_PHONE : [0 1]
TARGET : [  1.   0.  nan]
REG_REGION_NOT_LIVE_REGION : [0 1]
REG_REGION_NOT_WORK_REGION : [0 1]
LIVE_REGION_NOT_WORK_REGION : [0 1]
REG_CITY_NOT_LIVE_CITY : [0 1]
REG_CITY_NOT_WORK_CITY : [0 1]
LIVE_CITY_NOT_WORK_CITY : [0 1]
EMERGENCYSTATE_MODE : ['No' nan 'Yes']


In [49]:
# convert Y and N into 1 an 0
a[['FLAG_OWN_CAR', 'FLAG_OWN_REALTY']] = a[['FLAG_OWN_CAR', 'FLAG_OWN_REALTY']].replace({'Y': 1, 'N': 0})
a['EMERGENCYSTATE_MODE'] = a['EMERGENCYSTATE_MODE'].replace({'Yes': 1, 'No': 0})

# confirm all FLAG type columns are 0 & 1
for col in flag_cols:
    print(col, ':', a[col].unique())

FLAG_CONT_MOBILE : [1 0]
FLAG_DOCUMENT_10 : [0]
FLAG_DOCUMENT_11 : [0 1]
FLAG_DOCUMENT_12 : [0]
FLAG_DOCUMENT_13 : [0 1]
FLAG_DOCUMENT_14 : [0 1]
FLAG_DOCUMENT_15 : [0 1]
FLAG_DOCUMENT_16 : [0 1]
FLAG_DOCUMENT_17 : [0 1]
FLAG_DOCUMENT_18 : [0 1]
FLAG_DOCUMENT_19 : [0 1]
FLAG_DOCUMENT_2 : [0]
FLAG_DOCUMENT_20 : [0 1]
FLAG_DOCUMENT_21 : [0 1]
FLAG_DOCUMENT_3 : [1 0]
FLAG_DOCUMENT_4 : [0 1]
FLAG_DOCUMENT_5 : [0 1]
FLAG_DOCUMENT_6 : [0 1]
FLAG_DOCUMENT_7 : [0 1]
FLAG_DOCUMENT_8 : [0 1]
FLAG_DOCUMENT_9 : [0 1]
FLAG_EMAIL : [0 1]
FLAG_EMP_PHONE : [1 0]
FLAG_MOBIL : [1]
FLAG_OWN_CAR : [0 1]
FLAG_OWN_REALTY : [1 0]
FLAG_PHONE : [1 0]
FLAG_WORK_PHONE : [0 1]
TARGET : [  1.   0.  nan]
REG_REGION_NOT_LIVE_REGION : [0 1]
REG_REGION_NOT_WORK_REGION : [0 1]
LIVE_REGION_NOT_WORK_REGION : [0 1]
REG_CITY_NOT_LIVE_CITY : [0 1]
REG_CITY_NOT_WORK_CITY : [0 1]
LIVE_CITY_NOT_WORK_CITY : [0 1]
EMERGENCYSTATE_MODE : [  0.  nan   1.]


In [50]:
# confirm all FLAG type columns are int64 or float64
a[flag_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 100002 to 172574
Data columns (total 36 columns):
FLAG_CONT_MOBILE               20000 non-null int64
FLAG_DOCUMENT_10               20000 non-null int64
FLAG_DOCUMENT_11               20000 non-null int64
FLAG_DOCUMENT_12               20000 non-null int64
FLAG_DOCUMENT_13               20000 non-null int64
FLAG_DOCUMENT_14               20000 non-null int64
FLAG_DOCUMENT_15               20000 non-null int64
FLAG_DOCUMENT_16               20000 non-null int64
FLAG_DOCUMENT_17               20000 non-null int64
FLAG_DOCUMENT_18               20000 non-null int64
FLAG_DOCUMENT_19               20000 non-null int64
FLAG_DOCUMENT_2                20000 non-null int64
FLAG_DOCUMENT_20               20000 non-null int64
FLAG_DOCUMENT_21               20000 non-null int64
FLAG_DOCUMENT_3                20000 non-null int64
FLAG_DOCUMENT_4                20000 non-null int64
FLAG_DOCUMENT_5                20000 non-null int64
F

In [51]:
# the target column has nan because a_test doesn't have a value for that column

In [52]:
a['EMERGENCYSTATE_MODE'].isna().sum()

9342

In [53]:
a['EMERGENCYSTATE_MODE'].notna().sum()

10658

In [54]:
# remove adjusted cols from 'cols to clean'
cols_to_clean = [x for x in cols_to_clean if x not in flag_cols]
len(cols_to_clean)

86

#### Categorical Columns (cat1, cat2, cat3, ...)

In [55]:
# find all NAME and TYPE columns using their column headers

name_cols = [x for x in a_cols if 'NAME' in x]
type_cols = [x for x in a_cols if 'TYPE' in x]


cat_cols = name_cols + type_cols

cat_cols += ['CODE_GENDER', 
            'WEEKDAY_APPR_PROCESS_START', 
            'FONDKAPREMONT_MODE', 
            'WALLSMATERIAL_MODE',
            'REGION_RATING_CLIENT',
            'REGION_RATING_CLIENT_W_CITY']

cat_cols = list(dict.fromkeys(cat_cols)) # remove duplicates
cat_cols

['NAME_CONTRACT_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'NAME_INCOME_TYPE',
 'NAME_TYPE_SUITE',
 'HOUSETYPE_MODE',
 'OCCUPATION_TYPE',
 'ORGANIZATION_TYPE',
 'CODE_GENDER',
 'WEEKDAY_APPR_PROCESS_START',
 'FONDKAPREMONT_MODE',
 'WALLSMATERIAL_MODE',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY']

In [56]:
# print NAME or TYPE columns and the num of unique values
for col in cat_cols:
    print(col, ':', len(a[col].unique()))

NAME_CONTRACT_TYPE : 2
NAME_EDUCATION_TYPE : 5
NAME_FAMILY_STATUS : 5
NAME_HOUSING_TYPE : 6
NAME_INCOME_TYPE : 7
NAME_TYPE_SUITE : 8
HOUSETYPE_MODE : 4
OCCUPATION_TYPE : 19
ORGANIZATION_TYPE : 58
CODE_GENDER : 2
WEEKDAY_APPR_PROCESS_START : 7
FONDKAPREMONT_MODE : 5
WALLSMATERIAL_MODE : 8
REGION_RATING_CLIENT : 3
REGION_RATING_CLIENT_W_CITY : 3


In [57]:
# print NAME or TYPE columns and their unique values
for col in cat_cols:
    print(col, ':', a[col].unique())

NAME_CONTRACT_TYPE : ['Cash loans' 'Revolving loans']
NAME_EDUCATION_TYPE : ['Secondary / secondary special' 'Higher education' 'Incomplete higher'
 'Lower secondary' 'Academic degree']
NAME_FAMILY_STATUS : ['Single / not married' 'Married' 'Civil marriage' 'Widow' 'Separated']
NAME_HOUSING_TYPE : ['House / apartment' 'Rented apartment' 'With parents'
 'Municipal apartment' 'Office apartment' 'Co-op apartment']
NAME_INCOME_TYPE : ['Working' 'State servant' 'Commercial associate' 'Pensioner' 'Unemployed'
 'Businessman' 'Student']
NAME_TYPE_SUITE : ['Unaccompanied' 'Family' 'Spouse, partner' 'Children' 'Other_A' nan
 'Other_B' 'Group of people']
HOUSETYPE_MODE : ['block of flats' nan 'terraced house' 'specific housing']
OCCUPATION_TYPE : ['Laborers' 'Core staff' 'Accountants' 'Managers' nan 'Drivers'
 'Sales staff' 'Cleaning staff' 'Cooking staff' 'Private service staff'
 'Medicine staff' 'Security staff' 'High skill tech staff'
 'Waiters/barmen staff' 'Low-skill Laborers' 'Realty agents

In [58]:
a[cat_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 100002 to 172574
Data columns (total 15 columns):
NAME_CONTRACT_TYPE             20000 non-null object
NAME_EDUCATION_TYPE            20000 non-null object
NAME_FAMILY_STATUS             20000 non-null object
NAME_HOUSING_TYPE              20000 non-null object
NAME_INCOME_TYPE               20000 non-null object
NAME_TYPE_SUITE                19775 non-null object
HOUSETYPE_MODE                 10082 non-null object
OCCUPATION_TYPE                13699 non-null object
ORGANIZATION_TYPE              20000 non-null object
CODE_GENDER                    20000 non-null object
WEEKDAY_APPR_PROCESS_START     20000 non-null object
FONDKAPREMONT_MODE             6367 non-null object
WALLSMATERIAL_MODE             9976 non-null object
REGION_RATING_CLIENT           20000 non-null int64
REGION_RATING_CLIENT_W_CITY    20000 non-null int64
dtypes: int64(2), object(13)
memory usage: 2.4+ MB


In [59]:
# convert cat_cols into categorical data
for col in cat_cols:
    a[col] = pd.Categorical(a[col])

a[cat_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 100002 to 172574
Data columns (total 15 columns):
NAME_CONTRACT_TYPE             20000 non-null category
NAME_EDUCATION_TYPE            20000 non-null category
NAME_FAMILY_STATUS             20000 non-null category
NAME_HOUSING_TYPE              20000 non-null category
NAME_INCOME_TYPE               20000 non-null category
NAME_TYPE_SUITE                19775 non-null category
HOUSETYPE_MODE                 10082 non-null category
OCCUPATION_TYPE                13699 non-null category
ORGANIZATION_TYPE              20000 non-null category
CODE_GENDER                    20000 non-null category
WEEKDAY_APPR_PROCESS_START     20000 non-null category
FONDKAPREMONT_MODE             6367 non-null category
WALLSMATERIAL_MODE             9976 non-null category
REGION_RATING_CLIENT           20000 non-null category
REGION_RATING_CLIENT_W_CITY    20000 non-null category
dtypes: category(15)
memory usage: 455.7 KB


In [60]:
# remove adjusted cols from 'cols to clean'

cols_to_clean = [x for x in cols_to_clean if x not in cat_cols]
len(cols_to_clean)

71

#### Numerical Columns

In [61]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

num_cols = a.select_dtypes(include=numerics).columns.tolist()
num_cols = [x for x in cols_to_clean if x in num_cols]

a[num_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 100002 to 172574
Data columns (total 70 columns):
AMT_ANNUITY                     19990 non-null float64
AMT_CREDIT                      20000 non-null float64
AMT_GOODS_PRICE                 19993 non-null float64
AMT_INCOME_TOTAL                20000 non-null float64
AMT_REQ_CREDIT_BUREAU_DAY       17417 non-null float64
AMT_REQ_CREDIT_BUREAU_HOUR      17417 non-null float64
AMT_REQ_CREDIT_BUREAU_MON       17417 non-null float64
AMT_REQ_CREDIT_BUREAU_QRT       17417 non-null float64
AMT_REQ_CREDIT_BUREAU_WEEK      17417 non-null float64
AMT_REQ_CREDIT_BUREAU_YEAR      17417 non-null float64
APARTMENTS_AVG                  9990 non-null float64
APARTMENTS_MEDI                 9990 non-null float64
APARTMENTS_MODE                 9990 non-null float64
BASEMENTAREA_AVG                8460 non-null float64
BASEMENTAREA_MEDI               8460 non-null float64
BASEMENTAREA_MODE               8460 non-null float64
CNT_CHILDRE

In [62]:
# remove num_cols from cols_to_clean
cols_to_clean = [x for x in cols_to_clean if x not in num_cols]

#### Remaining Columns

In [63]:
cols_to_clean

['TRAIN_TEST']

In [64]:
a[cols_to_clean].head()

,TRAIN_TEST
SK_ID_CURR,
100002,train
100003,train
100004,train
100006,train
100007,train


Cleaning Completed for 'a' df

#### Split a back into a_train and a_test

### <font color='green'>2.3.2) Bureau</font>

In [65]:
b.head()

,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
SK_ID_CURR,,,,,,,,,,,,,,,,
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [66]:
b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 215354 to 215641
Data columns (total 16 columns):
SK_ID_BUREAU              10000 non-null int64
CREDIT_ACTIVE             10000 non-null object
CREDIT_CURRENCY           10000 non-null object
DAYS_CREDIT               10000 non-null int64
CREDIT_DAY_OVERDUE        10000 non-null int64
DAYS_CREDIT_ENDDATE       9369 non-null float64
DAYS_ENDDATE_FACT         6229 non-null float64
AMT_CREDIT_MAX_OVERDUE    3595 non-null float64
CNT_CREDIT_PROLONG        10000 non-null int64
AMT_CREDIT_SUM            10000 non-null float64
AMT_CREDIT_SUM_DEBT       8460 non-null float64
AMT_CREDIT_SUM_LIMIT      6550 non-null float64
AMT_CREDIT_SUM_OVERDUE    10000 non-null float64
CREDIT_TYPE               10000 non-null object
DAYS_CREDIT_UPDATE        10000 non-null int64
AMT_ANNUITY               1834 non-null float64
dtypes: float64(8), int64(5), object(3)
memory usage: 1.3+ MB


In [67]:
# print columns and the num of unique values

for col in b.columns:
    print(col, ':', len(b[col].unique()))

SK_ID_BUREAU : 10000
CREDIT_ACTIVE : 4
CREDIT_CURRENCY : 3
DAYS_CREDIT : 2698
CREDIT_DAY_OVERDUE : 40
DAYS_CREDIT_ENDDATE : 3930
DAYS_ENDDATE_FACT : 2252
AMT_CREDIT_MAX_OVERDUE : 757
CNT_CREDIT_PROLONG : 5
AMT_CREDIT_SUM : 4842
AMT_CREDIT_SUM_DEBT : 2526
AMT_CREDIT_SUM_LIMIT : 400
AMT_CREDIT_SUM_OVERDUE : 41
CREDIT_TYPE : 10
DAYS_CREDIT_UPDATE : 2063
AMT_ANNUITY : 643


In [68]:
# print columns and their unique values

for col in b.columns:
    print(col, ':', b[col].unique())

SK_ID_BUREAU : [5714462 5714463 5714464 ..., 5726629 5726630 5726631]
CREDIT_ACTIVE : ['Closed' 'Active' 'Sold' 'Bad debt']
CREDIT_CURRENCY : ['currency 1' 'currency 2' 'currency 4']
DAYS_CREDIT : [ -497  -208  -203 ..., -2907  -881 -1020]
CREDIT_DAY_OVERDUE : [   0 2603    6   30 2156  496  186 2264   41    8   71   13  252 1201   12
   38 1777  150   18   37   64 2050    9    4   29   22   49 1930   34  951
  936  664   45 2625  837   21   15   31 1715   19]
DAYS_CREDIT_ENDDATE : [ -153.  1075.   528. ..., -1055.  5210.  2683.]
DAYS_ENDDATE_FACT : [ -153.    nan -1710. ..., -1356.  -981.  -623.]
AMT_CREDIT_MAX_OVERDUE : [             nan   7.76745000e+04   0.00000000e+00   1.49850000e+04
   3.10500000e+02   2.04932700e+04   8.88210000e+04   1.35000000e+03
   3.98052000e+03   1.80000000e+03   1.82932650e+04   1.26180000e+04
   4.05000000e+01   6.00750000e+03   1.12500000e+04   6.49242000e+03
   3.28500000e+03   1.70478000e+03   3.72789000e+03   1.35900000e+04
   2.18920500e+03   1.322

AMT_CREDIT_SUM_OVERDUE : [  0.00000000e+00   2.31525000e+02   2.88000000e+02   5.85000000e+01
   5.04000000e+02   1.69582500e+05   3.60000000e+01   5.40000000e+02
   1.71000000e+03   4.94100000e+03   1.62000000e+03   4.24350000e+04
   3.49428060e+05   8.55000000e+02   2.24100000e+04   1.29150000e+03
   3.52350000e+03   2.29500000e+02   4.50000000e+01   3.51000000e+02
   6.07500000e+03   1.35000000e+01   1.13046750e+04   7.21800000e+02
   6.16500000e+02   4.91850000e+03   2.30985000e+04   4.09500000e+02
   1.51200000e+03   4.27500000e+02   7.60500000e+02   1.72048500e+05
   6.36483600e+04   5.65110000e+02   1.21500000e+02   8.41500000e+00
   9.00000000e+00   1.61845200e+04   4.95000000e+02   5.85000000e+02
   8.19000000e+02]
CREDIT_TYPE : ['Consumer credit' 'Credit card' 'Mortgage' 'Car loan' 'Microloan'
 'Loan for working capital replenishment' 'Loan for business development'
 'Real estate loan' 'Unknown type of loan' 'Another type of loan']
DAYS_CREDIT_UPDATE : [ -131   -20   -16 ...,

#### Categorical Columns

In [69]:
# convert cat_cols into categorical data

cat_cols = ['CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'CREDIT_TYPE']

for col in cat_cols:
    b[col] = pd.Categorical(b[col])

In [70]:
b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 215354 to 215641
Data columns (total 16 columns):
SK_ID_BUREAU              10000 non-null int64
CREDIT_ACTIVE             10000 non-null category
CREDIT_CURRENCY           10000 non-null category
DAYS_CREDIT               10000 non-null int64
CREDIT_DAY_OVERDUE        10000 non-null int64
DAYS_CREDIT_ENDDATE       9369 non-null float64
DAYS_ENDDATE_FACT         6229 non-null float64
AMT_CREDIT_MAX_OVERDUE    3595 non-null float64
CNT_CREDIT_PROLONG        10000 non-null int64
AMT_CREDIT_SUM            10000 non-null float64
AMT_CREDIT_SUM_DEBT       8460 non-null float64
AMT_CREDIT_SUM_LIMIT      6550 non-null float64
AMT_CREDIT_SUM_OVERDUE    10000 non-null float64
CREDIT_TYPE               10000 non-null category
DAYS_CREDIT_UPDATE        10000 non-null int64
AMT_ANNUITY               1834 non-null float64
dtypes: category(3), float64(8), int64(5)
memory usage: 1.1 MB


### <font color='green'>2.3.3) Bureau Balance</font>

In [71]:
bb.head()

,MONTHS_BALANCE,STATUS
SK_ID_BUREAU,,
5715448,0,C
5715448,-1,C
5715448,-2,C
5715448,-3,C
5715448,-4,C


In [72]:
bb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 5715448 to 5720034
Data columns (total 2 columns):
MONTHS_BALANCE    10000 non-null int64
STATUS            10000 non-null object
dtypes: int64(1), object(1)
memory usage: 234.4+ KB


In [73]:
# print columns and the num of unique values
for col in bb.columns:
    print(col, ':', len(bb[col].unique()))

MONTHS_BALANCE : 97
STATUS : 6


In [74]:
# print columns and their unique values
for col in bb.columns:
    print(col, ':', bb[col].unique())

MONTHS_BALANCE : [  0  -1  -2  -3  -4  -5  -6  -7  -8  -9 -10 -11 -12 -13 -14 -15 -16 -17
 -18 -19 -20 -21 -22 -23 -24 -25 -26 -27 -28 -29 -30 -31 -32 -33 -34 -35
 -36 -37 -38 -39 -40 -41 -42 -43 -44 -45 -46 -47 -48 -49 -50 -51 -52 -53
 -54 -55 -56 -57 -58 -59 -60 -61 -62 -63 -64 -65 -66 -67 -68 -69 -70 -71
 -72 -73 -74 -75 -76 -77 -78 -79 -80 -81 -82 -83 -84 -85 -86 -87 -88 -89
 -90 -91 -92 -93 -94 -95 -96]
STATUS : ['C' '0' 'X' '1' '2' '3']


In [75]:
# convert cat_cols into categorical data
bb['STATUS'] = pd.Categorical(bb['STATUS'])

In [76]:
bb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 5715448 to 5720034
Data columns (total 2 columns):
MONTHS_BALANCE    10000 non-null int64
STATUS            10000 non-null category
dtypes: category(1), int64(1)
memory usage: 166.2 KB


### <font color='green'>2.3.4) Credit Card Balance</font>

In [77]:
ccb.head()

,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
SK_ID_PREV,,,,,,,,,,,,,,,,,,,,,
2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [78]:
ccb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 2562384 to 1669462
Data columns (total 22 columns):
SK_ID_CURR                    10000 non-null int64
MONTHS_BALANCE                10000 non-null int64
AMT_BALANCE                   10000 non-null float64
AMT_CREDIT_LIMIT_ACTUAL       10000 non-null int64
AMT_DRAWINGS_ATM_CURRENT      7933 non-null float64
AMT_DRAWINGS_CURRENT          10000 non-null float64
AMT_DRAWINGS_OTHER_CURRENT    7933 non-null float64
AMT_DRAWINGS_POS_CURRENT      7933 non-null float64
AMT_INST_MIN_REGULARITY       9674 non-null float64
AMT_PAYMENT_CURRENT           7952 non-null float64
AMT_PAYMENT_TOTAL_CURRENT     10000 non-null float64
AMT_RECEIVABLE_PRINCIPAL      10000 non-null float64
AMT_RECIVABLE                 10000 non-null float64
AMT_TOTAL_RECEIVABLE          10000 non-null float64
CNT_DRAWINGS_ATM_CURRENT      7933 non-null float64
CNT_DRAWINGS_CURRENT          10000 non-null int64
CNT_DRAWINGS_OTHER_CURRENT    7933 non-null float

In [79]:
# print columns and the num of unique values
for col in ccb.columns:
    print(col, ':', len(ccb[col].unique()))

SK_ID_CURR : 9520
MONTHS_BALANCE : 69
AMT_BALANCE : 4495
AMT_CREDIT_LIMIT_ACTUAL : 55
AMT_DRAWINGS_ATM_CURRENT : 166
AMT_DRAWINGS_CURRENT : 1231
AMT_DRAWINGS_OTHER_CURRENT : 31
AMT_DRAWINGS_POS_CURRENT : 1067
AMT_INST_MIN_REGULARITY : 2527
AMT_PAYMENT_CURRENT : 2990
AMT_PAYMENT_TOTAL_CURRENT : 1680
AMT_RECEIVABLE_PRINCIPAL : 4231
AMT_RECIVABLE : 4369
AMT_TOTAL_RECEIVABLE : 4369
CNT_DRAWINGS_ATM_CURRENT : 17
CNT_DRAWINGS_CURRENT : 52
CNT_DRAWINGS_OTHER_CURRENT : 5
CNT_DRAWINGS_POS_CURRENT : 53
CNT_INSTALMENT_MATURE_CUM : 113
NAME_CONTRACT_STATUS : 5
SK_DPD : 57
SK_DPD_DEF : 6


In [80]:
# print columns and their unique values
for col in ccb.columns:
    print(col, ':', ccb[col].unique())

SK_ID_CURR : [378907 363914 371185 ..., 397003 311063 289406]
MONTHS_BALANCE : [ -6  -1  -7  -4  -5  -3  -2 -19 -13 -18 -15 -12 -22 -16 -21 -14 -20 -17
 -11 -52 -54 -58 -53 -51 -50 -59 -56 -57 -60 -55 -26 -25 -24 -23 -67 -64
 -62 -63 -61 -65 -66 -10  -9 -88 -95 -86 -89 -91 -93 -87 -90 -92 -94 -85
 -72 -73 -70 -69 -68 -71 -74  -8 -27 -80 -79 -84 -83 -81 -82]
AMT_BALANCE : [  5.69700000e+01   6.39755550e+04   3.18152250e+04 ...,   8.71839000e+04
   8.50992300e+04   7.66159200e+04]
AMT_CREDIT_LIMIT_ACTUAL : [ 135000   45000  450000  225000  270000  585000  292500  180000   67500
       0  157500  360000  540000  558000  202500  247500  112500   90000
  720000  315000  900000   22500  675000  855000  382500  495000  337500
  765000  405000  108000  630000  810000  427500  261000  229500   27000
   81000  216000  103500   58500  468000  342000   94500  274500  234000
  130500  400500    4500   36000  306000 1125000   72000   54000   63000
  162000]
AMT_DRAWINGS_ATM_CURRENT : [  0.00000000e+

In [81]:
# categoricial column(s)
ccb['NAME_CONTRACT_STATUS'] = pd.Categorical(ccb['NAME_CONTRACT_STATUS'])

In [82]:
ccb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 2562384 to 1669462
Data columns (total 22 columns):
SK_ID_CURR                    10000 non-null int64
MONTHS_BALANCE                10000 non-null int64
AMT_BALANCE                   10000 non-null float64
AMT_CREDIT_LIMIT_ACTUAL       10000 non-null int64
AMT_DRAWINGS_ATM_CURRENT      7933 non-null float64
AMT_DRAWINGS_CURRENT          10000 non-null float64
AMT_DRAWINGS_OTHER_CURRENT    7933 non-null float64
AMT_DRAWINGS_POS_CURRENT      7933 non-null float64
AMT_INST_MIN_REGULARITY       9674 non-null float64
AMT_PAYMENT_CURRENT           7952 non-null float64
AMT_PAYMENT_TOTAL_CURRENT     10000 non-null float64
AMT_RECEIVABLE_PRINCIPAL      10000 non-null float64
AMT_RECIVABLE                 10000 non-null float64
AMT_TOTAL_RECEIVABLE          10000 non-null float64
CNT_DRAWINGS_ATM_CURRENT      7933 non-null float64
CNT_DRAWINGS_CURRENT          10000 non-null int64
CNT_DRAWINGS_OTHER_CURRENT    7933 non-null float

### <font color='green'>2.3.5) Previous Application</font>

In [83]:
pa.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
SK_ID_PREV,,,,,,,,,,,,,,,,,,,,,
2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
pa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 2030495 to 2744934
Data columns (total 36 columns):
SK_ID_CURR                     10000 non-null int64
NAME_CONTRACT_TYPE             10000 non-null object
AMT_ANNUITY                    7915 non-null float64
AMT_APPLICATION                10000 non-null float64
AMT_CREDIT                     10000 non-null float64
AMT_DOWN_PAYMENT               4954 non-null float64
AMT_GOODS_PRICE                7878 non-null float64
WEEKDAY_APPR_PROCESS_START     10000 non-null object
HOUR_APPR_PROCESS_START        10000 non-null int64
FLAG_LAST_APPL_PER_CONTRACT    10000 non-null object
NFLAG_LAST_APPL_IN_DAY         10000 non-null int64
RATE_DOWN_PAYMENT              4954 non-null float64
RATE_INTEREST_PRIMARY          37 non-null float64
RATE_INTEREST_PRIVILEGED       37 non-null float64
NAME_CASH_LOAN_PURPOSE         10000 non-null object
NAME_CONTRACT_STATUS           10000 non-null object
DAYS_DECISION                  10000 non

In [85]:
# print columns and the num of unique values
for col in pa.columns:
    print(col, ':', len(pa[col].unique()))

SK_ID_CURR : 9734
NAME_CONTRACT_TYPE : 4
AMT_ANNUITY : 6598
AMT_APPLICATION : 3884
AMT_CREDIT : 5313
AMT_DOWN_PAYMENT : 1380
AMT_GOODS_PRICE : 3885
WEEKDAY_APPR_PROCESS_START : 7
HOUR_APPR_PROCESS_START : 22
FLAG_LAST_APPL_PER_CONTRACT : 2
NFLAG_LAST_APPL_IN_DAY : 2
RATE_DOWN_PAYMENT : 2240
RATE_INTEREST_PRIMARY : 16
RATE_INTEREST_PRIVILEGED : 8
NAME_CASH_LOAN_PURPOSE : 23
NAME_CONTRACT_STATUS : 4
DAYS_DECISION : 2544
NAME_PAYMENT_TYPE : 4
CODE_REJECT_REASON : 8
NAME_TYPE_SUITE : 8
NAME_CLIENT_TYPE : 4
NAME_GOODS_CATEGORY : 24
NAME_PORTFOLIO : 5
NAME_PRODUCT_TYPE : 3
CHANNEL_TYPE : 8
SELLERPLACE_AREA : 697
NAME_SELLER_INDUSTRY : 11
CNT_PAYMENT : 27
NAME_YIELD_GROUP : 5
PRODUCT_COMBINATION : 18
DAYS_FIRST_DRAWING : 315
DAYS_FIRST_DUE : 2337
DAYS_LAST_DUE_1ST_VERSION : 2711
DAYS_LAST_DUE : 2106
DAYS_TERMINATION : 2040
NFLAG_INSURED_ON_APPROVAL : 3


In [86]:
# print columns and their unique values
for col in pa.columns:
    print(col, ':', pa[col].unique())

SK_ID_CURR : [271877 108129 122040 ..., 202061 233764 277168]
NAME_CONTRACT_TYPE : ['Consumer loans' 'Cash loans' 'Revolving loans' 'XNA']
AMT_ANNUITY : [  1730.43   25188.615  15060.735 ...,   2791.935  42176.295   7395.48 ]
AMT_APPLICATION : [  17145.  607500.  112500. ...,  283500.  131400.   16875.]
AMT_CREDIT : [   17145.    679671.    136444.5 ...,  1211503.5   140071.5    14625. ]
AMT_DOWN_PAYMENT : [     0.       nan  12649.5 ...,   7164.    2340.   12307.5]
AMT_GOODS_PRICE : [  17145.  607500.  112500. ...,  283500.  131400.   16875.]
WEEKDAY_APPR_PROCESS_START : ['SATURDAY' 'THURSDAY' 'TUESDAY' 'MONDAY' 'FRIDAY' 'SUNDAY' 'WEDNESDAY']
HOUR_APPR_PROCESS_START : [15 11  7  9  8 10 12 13 14 16  6  4  5 19 17 18 20 22 21  3  1  2]
FLAG_LAST_APPL_PER_CONTRACT : ['Y' 'N']
NFLAG_LAST_APPL_IN_DAY : [1 0]
RATE_DOWN_PAYMENT : [ 0.                 nan  0.10397129 ...,  0.09947447  0.14587341
  0.14521212]
RATE_INTEREST_PRIMARY : [ 0.1828318          nan  0.19691431  0.18913635  0.6956675

In [87]:
# cols to convert type

cat_cols = ['NAME_CONTRACT_TYPE', 
            'WEEKDAY_APPR_PROCESS_START', 
            'NAME_CASH_LOAN_PURPOSE', 
            'NAME_CONTRACT_STATUS',
            'NAME_PAYMENT_TYPE', 
            'CODE_REJECT_REASON', 
            'NAME_TYPE_SUITE', 
            'NAME_CLIENT_TYPE', 
            'NAME_GOODS_CATEGORY',
            'NAME_PORTFOLIO', 
            'NAME_PRODUCT_TYPE', 
            'CHANNEL_TYPE', 
            'NAME_SELLER_INDUSTRY', 
            'NAME_YIELD_GROUP', 
            'PRODUCT_COMBINATION']

flag_cols = ['FLAG_LAST_APPL_PER_CONTRACT', 
             'NFLAG_LAST_APPL_IN_DAY', 
             'NFLAG_INSURED_ON_APPROVAL']


In [88]:
# categoricial column(s)
pa[cat_cols] = pd.Categorical(pa[cat_cols])

In [89]:
# convert Y and N into 1 an 0
pa[flag_cols] = pa[flag_cols].replace({'Y': 1, 'N': 0})
pa[flag_cols].head()

,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,NFLAG_INSURED_ON_APPROVAL
SK_ID_PREV,,,
2030495,1,1,0.0
2802425,1,1,1.0
2523466,1,1,1.0
2819243,1,1,1.0
1784265,1,1,NaN


In [90]:
pa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 2030495 to 2744934
Data columns (total 36 columns):
SK_ID_CURR                     10000 non-null int64
NAME_CONTRACT_TYPE             10000 non-null object
AMT_ANNUITY                    7915 non-null float64
AMT_APPLICATION                10000 non-null float64
AMT_CREDIT                     10000 non-null float64
AMT_DOWN_PAYMENT               4954 non-null float64
AMT_GOODS_PRICE                7878 non-null float64
WEEKDAY_APPR_PROCESS_START     10000 non-null object
HOUR_APPR_PROCESS_START        10000 non-null int64
FLAG_LAST_APPL_PER_CONTRACT    10000 non-null int64
NFLAG_LAST_APPL_IN_DAY         10000 non-null int64
RATE_DOWN_PAYMENT              4954 non-null float64
RATE_INTEREST_PRIMARY          37 non-null float64
RATE_INTEREST_PRIVILEGED       37 non-null float64
NAME_CASH_LOAN_PURPOSE         10000 non-null object
NAME_CONTRACT_STATUS           10000 non-null object
DAYS_DECISION                  10000 non-

### <font color='green'>2.3.6) Sample Submission</font>

In [91]:
ss.head()

,TARGET
SK_ID_CURR,
100001,0.5
100005,0.5
100013,0.5
100028,0.5
100038,0.5


In [92]:
ss.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 100001 to 172574
Data columns (total 1 columns):
TARGET    10000 non-null float64
dtypes: float64(1)
memory usage: 156.2 KB


## <font color='blue'>2.4) Missing Values</font>

### <font color='green'>2.4.1) Application_test|train</font>

In [93]:
a[a.columns[a.isnull().any()]].head()

,AMT_ANNUITY,AMT_GOODS_PRICE,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,...,OWN_CAR_AGE,TARGET,TOTALAREA_MODE,WALLSMATERIAL_MODE,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_AVG,YEARS_BUILD_MEDI,YEARS_BUILD_MODE
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,24700.5,351000.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0247,0.0250,...,NaN,1.0,0.0149,"Stone, brick",0.9722,0.9722,0.9722,0.6192,0.6243,0.6341
100003,35698.5,1129500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0959,0.0968,...,NaN,0.0,0.0714,Block,0.9851,0.9851,0.9851,0.7960,0.7987,0.8040
100004,6750.0,135000.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,26.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100006,29686.5,297000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100007,21865.5,513000.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
# info for columns with missing data
a[a.columns[a.isnull().any()]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 100002 to 172574
Data columns (total 66 columns):
AMT_ANNUITY                     19990 non-null float64
AMT_GOODS_PRICE                 19993 non-null float64
AMT_REQ_CREDIT_BUREAU_DAY       17417 non-null float64
AMT_REQ_CREDIT_BUREAU_HOUR      17417 non-null float64
AMT_REQ_CREDIT_BUREAU_MON       17417 non-null float64
AMT_REQ_CREDIT_BUREAU_QRT       17417 non-null float64
AMT_REQ_CREDIT_BUREAU_WEEK      17417 non-null float64
AMT_REQ_CREDIT_BUREAU_YEAR      17417 non-null float64
APARTMENTS_AVG                  9990 non-null float64
APARTMENTS_MEDI                 9990 non-null float64
APARTMENTS_MODE                 9990 non-null float64
BASEMENTAREA_AVG                8460 non-null float64
BASEMENTAREA_MEDI               8460 non-null float64
BASEMENTAREA_MODE               8460 non-null float64
COMMONAREA_AVG                  6146 non-null float64
COMMONAREA_MEDI                 6146 non-null float64
COMMONAREA_MO

### <font color='green'>2.4.2) Bureau</font>

In [ ]:
# info for columns with missing data
b[b.columns[b.isnull().any()]].info()

### <font color='green'>2.4.3) Bureau Balance</font>

In [ ]:
# info for columns with missing data
bb[bb.columns[bb.isnull().any()]].info()
# no missing values

### <font color='green'>2.4.4) Credit Card Balance</font>

In [ ]:
# info for columns with missing data
ccb[ccb.columns[ccb.isnull().any()]].info()

### <font color='green'>2.4.5) Previous Application</font>

In [ ]:
# info for columns with missing data
pa[pa.columns[pa.isnull().any()]].info()

### <font color='green'>2.4.6) Sample Submission</font>

In [ ]:
# info for columns with missing data
ss[ss.columns[ss.isnull().any()]].info()
# no missing values

## <font color='blue'>2.5) Outlier Treatment</font>

## <font color='blue'>2.6) Feature Engineering</font>

## <font color='blue'>2.6) Combine Datasets</font>

### <font color='green'>2.6.1) Bureau + Bureau Balances</font>

In [ ]:
b.head()

In [ ]:
bb.head()

In [ ]:
# compare column headers

cols1 = list(b.columns)
cols1.append(b.index.name)

cols2 = list(bb.columns)
cols2.append(bb.index.name)

new_cols = list(set(cols1).intersection(cols2))
new_cols

In [ ]:
b_bb = pd.merge(b, bb, how='inner', on='SK_ID_BUREAU', sort=True)
b_bb = b_bb.set_index('SK_ID_BUREAU')
b_bb.head()

### June 10
# lastest
# transpose MONTHS_BALANCE
# average
# most occuring

In [ ]:
b_bb.info()

# 3) Split Data v1

MENTOR: Generally data is not previously split into train and test dfs, but since that is predifined, the next two steps will just split along the original split. Then the will be split into X and y dfs.

The problem with this, is that you cant use the y_test df. It only has NaN values.

## <font color='blue'>3.1) Split Data into Train/Test</font>

In [ ]:
# a_train data
a_train = a[a['TRAIN_TEST']=='train']
a_train = a_train.drop(columns=['TRAIN_TEST'])
a_train.head()

In [ ]:
# a_test data

a_test = a[a['TRAIN_TEST']=='test']
a_test = a_test.drop(columns=['TRAIN_TEST'])
a_test.head()

## <font color='blue'>3.2) Split Data into XY</font>

In [ ]:
# create X_train
X_train = a_train.drop(['TARGET'], axis=1)
X_train.head()

In [ ]:
# create y_train
y_train = a_train['TARGET']
y_train.head()

In [ ]:
# create X_train
X_test = a_test.drop(['TARGET'], axis=1)
X_test.head()

In [ ]:
# create y_train
y_test = a_test['TARGET']
y_test.head()

# 3) Split Data v2

MENTOR: One second thought... The a_train df has values for X and for y. The a_test df has values for X, but not for y. One approach is to split the a_train df into train/test (70/30), then into X_train, y_train, X_test, and y_test. And then convert the a_test df into the X_pred and predict y_pred for the submission. What are your thoughts?

## <font color='blue'>3.1) Split Data into train_test and pred</font>

In [ ]:
# train_test data
train_test = a[a['TRAIN_TEST']=='train']
train_test = train_test.drop(columns=['TRAIN_TEST'])
train_test.head()

In [ ]:
# pred data
pred = a[a['TRAIN_TEST']=='test']
pred = pred.drop(columns=['TRAIN_TEST'])
pred.head()

## <font color='blue'>3.2) Split Data into XY</font>

In [ ]:
# create X
X = train_test.drop(['TARGET'], axis=1)
X.head()

In [ ]:
# create y_train
y = train_test['TARGET']
y.head()

## <font color='blue'>3.3) Split Data into Train/Test</font>

In [ ]:
# split df into train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
X_test.head()

In [ ]:
y_test.head()

In [ ]:
X_pred = pred.drop(['TARGET'], axis=1)
X_pred.head()

In [ ]:
y_pred = pred['TARGET']
y_pred.head()

# 4) Exploratory Data Analysis

## <font color='blue'>4.1) Correlation Matrix</font>

In [ ]:
# compute the correlation matrix
corr = X_train.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(10,10))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
_ = sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True, fmt='.2f')

corr

In [ ]:

corr_filter = pd.DataFrame(corr.stack()) # convert columns names into row values
corr_filter.reset_index(inplace=True) # convert the multi index into new cols
corr_filter.columns = ['var_1', 'var_2', 'corr'] # set column names

corr_filter['var_min'] = np.minimum(corr_filter['var_1'], corr_filter['var_2'])
corr_filter['var_max'] = np.maximum(corr_filter['var_1'], corr_filter['var_2'])
corr_filter['var_comb'] = corr_filter['var_min'] + '_+_' + corr_filter['var_max']
corr_filter = corr_filter.drop_duplicates('var_comb')
corr_filter = corr_filter[['var_comb','var_1', 'var_2', 'corr']]

corr_filter = corr_filter[corr_filter['var_1'] != corr_filter['var_2']] # remove a:a correlation
filt = 0.99 # set filter min for correlation value
corr_filter = corr_filter[(corr_filter['corr']>filt) 
                          | (corr_filter['corr']<-1*filt)] # apply filter

corr_filter['corr_abs'] = corr_filter['corr'].abs()
corr_filter = corr_filter.sort_values(by=['corr_abs'])
corr_filter = corr_filter.drop('corr_abs', axis=1)

corr_filter

In [ ]:
corr_final = corr_filter[['var_comb','corr']]
corr_final

In [ ]:
ax = sns.barplot(x='corr', y='var_comb', data=corr_final)

count = 0
for index, row in corr_final.iterrows():
    ax.text(abs(row['corr'])+.0001, count+0.25, 
        round(row['corr'],4), 
        ha='left',
        fontsize=10)
    count += 1

_ = plt.xlim([0.997,1.00])

## <font color='blue'>4.2) TARGET Distribution</font>

In [ ]:
y_train.value_counts()

In [ ]:
ax = sns.countplot(y_train)
_ = plt.title('TARGET Distribution')
_ = plt.ylabel('Count')
_ = plt.ylim([0, 8000])


_ = ax.text(0, 7000, 
        str(6460), 
        ha='center',
        fontsize=14, 
            fontweight='bold')

_ = ax.text(1, 1000, 
        str(540), 
        ha='center',
        fontsize=14, 
            fontweight='bold')

# 5) Variable Selection

## <font color='blue'>5.1) Encoding</font>

In [ ]:
cat_cols = list(X_train.select_dtypes(include='category').columns)
X_train[cat_cols].info()

In [ ]:
# print columns and the num of unique values

for col in cat_cols:
    print(col, ':', len(a_train[col].unique()))

In [ ]:
# print columns and their unique values

for col in cat_cols:
    print(col, ':', a_train[col].unique())
    print()

### <font color='green'>5.1.1) Label Encoding</font>

LABEL ENCODING: assign each unique category in a categorical variable with an integer. No new columns are created. Use for columns with only 2 categories.

In [ ]:
# label encode categorical columns

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder 

# create a label encoder object
le = LabelEncoder()
le_count = 0

# iterate through the columns
for col in X_train.columns:
    if X_train[col].dtype.name == 'category':
        
        # If 2 or fewer unique categories
        if len(list(X_train[col].unique())) <= 2:
            
            print(col)
            
            # Train on the training data
            le.fit(X_train[col])
            
            # Transform both training and testing data
            X_train[col] = le.transform(X_train[col])
            X_test[col] = le.transform(X_test[col])

            # Keep track of how many columns were label encoded
            le_count += 1

print('%d columns were label encoded.' % le_count)

### <font color='green'>5.1.2) One-Hot Encoding</font>

ONE-HOT ENCODING: create a new column for each unique category in a categorical variable. Each observation recieves a 1 in the column for its corresponding category and a 0 in all other new columns. Use for columns with 3+ categories.

In [ ]:
print('Number of Columns Before to One-Hot Encoding:')
print('X_train:', X_train.shape[1])
print('X_test:', X_test.shape[1])

In [ ]:
# one-hot encode categorical columns
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [ ]:
print('Number of Columns After to One-Hot Encoding:')
print('X_train:', X_train.shape[1])
print('X_test:', X_test.shape[1])

In [ ]:
# compare columns of X_train and X_test to make sure they are the same

train_cols = X_train.columns.tolist().sort()
test_cols = X_test.columns.tolist().sort()

if train_cols == test_cols:
    print('GOOD: The columns for training and testing are the same.')
else:
    print('BAD: The columns for training and testing are NOT the same.')

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
X_train.info()

## <font color='blue'>5.2) Information Value Analysis (IV)</font>

Information Value Analysis (IV) is a data exploration technique that helps determine which data columns in a dataset have predictive power or influence on the value of a specified dependent variable.

In [ ]:
#import libraries for IV

from scipy import stats
import pandas.core.algorithms as algos
import re
import traceback

In [ ]:
max_bin = 20
force_bin = 3

In [ ]:
# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

In [ ]:
def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

In [ ]:
def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(pd.Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [ ]:
final_iv, IV = data_vars(X_train, y_train)

In [ ]:
final_iv.head()

In [ ]:
IV.sort_values(['IV'], ascending=False)

In [ ]:
features = list(IV[(IV['IV'] >= 0.01) & (IV['IV'] <= 0.8)]['VAR_NAME'])
X2 = X_train[features]
X2.head()

In [ ]:
print('The number of columns in X2 after IV:', len(X2.columns.tolist()))

In [ ]:
X2.info()

## <font color='blue'>5.3) Variance Inflation Factor (VIF)</font>

Variance Inflation Factor (VIF) is the ratio of variance in a model with multiple terms, divided by the variance of a model with one term alone. It quantifies the severity of multicollinearity in an ordinary least squares regression analysis.

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
def iterate_vif(df, vif_threshold=5, max_vif=6):
  count = 0
  while max_vif > vif_threshold:
    count += 1
    print("Iteration # "+str(count))
    vif = pd.DataFrame()
    vif["VIFactor"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    vif["features"] = df.columns
    
    if vif['VIFactor'].max().round(1) > vif_threshold:
      print('Removing %s with VIF of %f' % (vif[vif['VIFactor'] == vif['VIFactor'].max()]['features'].values[0], 
                                            vif['VIFactor'].max().round(1)))
      df = df.drop(vif[vif['VIFactor'] == vif['VIFactor'].max()]['features'].values[0], axis=1)
      max_vif = vif['VIFactor'].max().round(1)
    else:
        print('Complete')
        return df, vif.sort_values('VIFactor').round(1)

In [ ]:
X3 = X2._get_numeric_data()
final_df, final_vif = iterate_vif(X3)

# Missing Data Error
# Need to Handle Missing Data

In [ ]:
final_df.head()

In [ ]:
final_cols = list(final_df.columns)
final_cols

## <font color='blue'>5.4) Final Training and Test Sets</font>

In [ ]:
X_train = X_train[final_cols]
X_train.head()

In [ ]:
### may need in the future
###
###

# align df so that each one has a 

# save the train labels for later
train_labels = a_train['TARGET']

# align the training and testing data, keep only columns present in both dataframes
a_train, a_test = a_train.align(a_test, 
                                join='inner',
                                axis = 1)

# add the target back in
a_train['TARGET'] = train_labels

print('Training Features shape: ', a_train.shape)
print('Testing Features shape: ', a_test.shape)

<div class="alert alert-block alert-info">
<b>Tip:</b> Use blue boxes (alert-info) for tips and notes. 
If it’s a note, you don’t have to include the word “Note”.
</div>

In [ ]:
### Check out Days_Employed in the pdf Gentle Introduction

## <font color='green'>Application Train</font>

In [ ]:
# explore application_train data
a_train.head()

In [ ]:
# histogram plot of target values

print('In the training data, target column, there are %s 0s and %s 1s.' \
      % (a_train[a_train.TARGET==0].shape[0], a_train[a_train.TARGET==1].shape[0]))

sns.set()
_ = sns.distplot(a_train['TARGET'], kde=False)
_ = plt.ylabel('COUNT')

Due to the unbalanced nature of the target data (11x as many loans repaid as were loans not repaid), the classes (0,1) will have to be weighted to balance out the amount of each class.

### Missing Data

In [ ]:
# function to calculate missing values per column

def miss_val_df(df):
    # Total missing values
    mis_val = df.isnull().sum()

    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})

    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
    mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)

    # Print some summary information
    print(str(mis_val_table_ren_columns.shape[0]) 
          + ' out of the ' 
          + str(df.shape[1]) 
          + ' total columns have missing values.')

    # Return the dataframe with missing information
    return mis_val_table_ren_columns

In [ ]:
miss_val_df(a_train).head(25)

In [ ]:
# column types
a_train.dtypes.value_counts()

In [ ]:
# number of unique values per category column
a_train.select_dtypes('object').apply(pd.Series.nunique, axis = 0).sort_values(ascending=False)

In [ ]:
cols = a_train.columns.tolist()
print(len(cols))
a_train = pd.get_dummies(a_train)
a_train.head()

# <font color='blue'>Data Storytelling</font>

# <font color='blue'>Modeling</font>